# _Transit Time Matrix Creation Program_

## Python Modules

In [ ]:
import time
import matplotli.pyplot as pl
import numpy as np
import multiprocessing as mp

## Created Modules

In [ ]:
import parametro
import auxfunctionsmodule as aux
import fortransubroutines as fortran

## Begining of the Transit Time Matrix Creation Program

In [ ]:
# This variable will tell fortran tha we want to do the modelling of the transit time matrix
regTTM = 1 

start_time = time.time()

## Velocity Model

### Loads and plots the velocity model that will be modeling

In [ ]:
C = aux.readbinaryfile(parametro.Nz,parametro.Nx,parametro.modelosuavizado)
aux.plotmodel(C, 'jet')

## Seismic Source

### Creates and plots the seismic source that will be used

In [ ]:
# Create the seismic source

fortran.wavelet(1,parametro.dt,1,parametro.f_corte)

In [ ]:
# Shows the seismic pulse

aux.plotgraphics(2,'wavelet_ricker.dat','k')
pl.show()

### Defines the source positions

#### Creates the file with the source positions

In [ ]:
lixo, fonte = np.loadtxt('wavelet_ricker.dat', unpack = True)
Nfonte      = np.size(fonte)

### Defines the source positions

#### Creates the file with the source positions

In [ ]:
if parametro.gera_pos_fonte: 
    aux.posicao_fonte(parametro.Nz,parametro.Nx,parametro.N_shot,parametro.Fx0,parametro.Fz0,parametro.SpaFonte)

#### Loads the file created

In [ ]:
Fx, Fz = np.loadtxt('posicoes_fonte.dat',dtype = 'int',unpack = True)
N_shot = np.size(Fx)

print(N_shot)

## Cerjan Condition

### Creates and plots the damping function 

In [ ]:
# Creates the damping function
func_amort = aux.amort(parametro.fat,parametro.nat)

In [ ]:
# plots the created function
aux.plotgraphics(1,'f_amort.dat','k')
pl.show()

## Acoustic Modeling - Transit Time Matrix Creation

### Here we are going to divide the problem in two cases: If we have only one shot and if we have multiple shots.

#### One shot

In [ ]:
# Case 1: Only one shot

if N_shot == 1:
     print("Fx =", Fx, "Fz =", Fz, "shot",N_shot)
    fortran.nucleomodelagem(parametro.Nz,parametro.Nx,parametro.Nt,\
                                parametro.h,parametro.dt,parametro.nat,\
                                N_shot,parametro.shotshow,\
                                Fx,Fz,fonte,parametro.Nsnap,regTTM,\
                                parametro.modelosuavizado,parametro.caminho_TTM,\
                                parametro.nome_prin,\
                                parametro.zr,)
    print(" shot= ",shot," Finalizado.")

#### Multiple shots

In [ ]:
# Case 2: More than one shot -> use parallelization

else:
     procs = []    
    for shot in np.arange(0,N_shot):
        proc = mp.Process(target=aux.modelagemparalela, args=(shot+1,Fx[shot],Fz[shot],fonte,regTTM))
        procs.append(proc)
        proc.start()
    
    for proc in procs:
        proc.join()

## Finishing the program

In [ ]:
elapsed_time_python = time.time() - start_time
print ("Tempo de processamento python = ", elapsed_time_python, "s")